In [1]:
from itertools import permutations
import numpy as np
import math
import random
from scipy.optimize import fsolve, root

In [2]:
def all_permutations(N, perm, i, vals=[-1, 1]):
    if i == N:
        yield perm
        return
    for val in vals:
        perm[i] = val
        yield from all_permutations(N, perm.copy(), i+1, vals)

def neighbour_sum(perm):
    ns = 0
    N = len(perm)
    for i in range(N):
            cur, nex = perm[i], perm[(i+1)%N]
            ns += cur * nex
    return ns / N

def self_sum(perm):
    return np.sum(perm) / len(perm)

def perm_eqns(nsum, ssum, lmd, mu, beta):
    base = math.exp(lmd*nsum + mu*ssum + beta - 1)
    return np.array([base*nsum, base*ssum, base])

def next_random_permutation(N, vals, lmd=1, mu=1, beta=1, eps=100): # choose permutations intelligently
    while True:
        perm = np.array(random.choices(vals, k=N))
        ns, ss = neighbour_sum(perm), self_sum(perm)
        eq = perm_eqns(ns, ss, lmd, mu, beta)
        if random.random() < eq[2] * eps:      # epsilon artificially increases the odds of selection (for speed)
            return perm, eq

def target_eqns(x, N, C, M, seed=1, nperms=100, vals=[-1, 1]):
    lmd, mu, beta = x
    eqns = [0] * 3
    random.seed(seed)
    for _ in range(nperms):
        perm, eq = next_random_permutation(N, vals, lmd, mu, beta)
        # print(perm, eq)
        eqns = eqns + eq
        # print(eqns)
    targets = np.array([C, M, 1])
    # print(eqns, targets, eqns-targets)
    return eqns - targets

In [3]:
N = 30
C, M = 1, -1

#sol = fsolve(target_eqns, (1., 1., 1.), args=(N, C, M), full_output=True, maxfev=1000)
#sol

In [4]:
def monte_carlo_search(N, C, M, trials=5):
    params = np.array([1, 1, 1])
    results = np.array([0, 0, 0])
    for tr in range(1, trials+1):
        sol = fsolve(target_eqns, params, args=(N, C, M, tr), maxfev=1000)
        print("Iteration {}: results - {}".format(tr, sol))
        results = results + sol
    results = results / trials
    return results

sol = monte_carlo_search(N, C, M)
sol

/home/amogh/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


Iteration 1: results - [ 10.29358589  -9.64492335 -12.33032944]


/home/amogh/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


Iteration 2: results - [ 12.06586313 -10.40452666 -14.11601446]
Iteration 3: results - [ 13.66672172  -5.98989125 -10.87596331]
Iteration 4: results - [ 13.85275111  -7.65474419 -13.87751361]
Iteration 5: results - [ 11.07236323 -10.15713849 -13.23465273]


array([ 12.19025702,  -8.77024479, -12.88689471])

In [11]:
perms = [[-1] * N]
perms.append([1] * N)
for _ in range(28):
    perms.append(next_random_permutation(N, [-1, 1])[0])

for perm in perms:
    ns, ss = neighbour_sum(perm), self_sum(perm)
    print(perm, perm_eqns(ns, ss, sol[0], sol[1], sol[2])[2], sep = "\t")

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]	1180.3981681192672
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]	2.8463656843264338e-05
[ 1  1 -1  1 -1 -1  1  1  1  1  1  1  1 -1  1  1 -1 -1 -1  1 -1  1 -1 -1
 -1  1  1  1  1 -1]	3.632190040349412e-07
[ 1 -1  1  1  1 -1 -1  1  1 -1  1  1  1 -1 -1 -1  1  1 -1  1  1  1 -1 -1
 -1  1 -1 -1 -1  1]	1.169547195919229e-06
[-1  1 -1  1  1 -1 -1  1  1  1  1 -1 -1  1 -1  1 -1  1  1 -1 -1 -1 -1 -1
 -1  1 -1 -1  1  1]	7.412739873002607e-07
[-1 -1 -1  1  1  1  1 -1 -1  1  1  1  1  1 -1 -1 -1  1  1 -1 -1 -1  1 -1
  1 -1 -1 -1  1 -1]	1.9131768690005132e-05
[ 1 -1 -1  1 -1  1  1 -1 -1 -1 -1  1 -1  1  1 -1 -1 -1  1  1  1  1 -1 -1
  1  1 -1 -1  1  1]	2.0986612630729762e-06
[ 1 -1 -1 -1 -1 -1  1 -1  1  1  1 -1 -1  1  1 -1  1 -1  1  1  1  1 -1 -1
  1 -1 -1 -1  1 -1]	7.412739873002607e-07
[ 1 -1 -1  1 -1 -1  1  1 -1  1 -1 -1  1  1 -1 -1  1  1